# Custom Loss
The SN² Solver tries to minimize the distance between the sample covariance
and the covariance induced by the model. There are more than one way to compute
this distance.

We will go through different methods for computing the distance between the sample covariance and the induced one.
These functions will be called the loss functions.
Let's start by taking the example from the [introduction](introduction.ipynb).


In [ ]:
import torch
from sn2_cuda import SN2Solver
from sn2_cuda.loss import LossBase, KullbackLeibler, Bhattacharyya, SquaredHellinger

device = torch.device("cuda")

struct = torch.tensor([
        [1, 1, 1, 0],
        [0, 1, 0, 1],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 1, 1],  # V_X
        [0, 0, 1, 0],  # V_BP
        [0, 0, 0, 1],  # V_BMI
        [0, 0, 0, 0],  # V_Y
    ], dtype=torch.bool)

sample_covariance = torch.tensor([
        [2,  3,  6,  8],
        [3,  7, 12, 16],
        [6, 12, 23, 30],
        [8, 16, 30, 41],
    ], device=device)

Before anything, we encapsulate our training code for multiple use.

In [11]:
def parametrize(pmDAG, max_iterations, min_error):
    optim = torch.optim.Adamax([pmDAG.weights], lr=0.001)

    for i in range(max_iterations):
        pmDAG.forward()
        error = pmDAG.loss().item()

        if error < min_error:
            break

        pmDAG.backward()
        optim.step()

        if i % (max_iterations / 10) == 0:
            print(f"iteration={i:<10} loss={error:<15.5}")
    else:
        print("Did not converge in the maximum number of iterations!")

We also configure the parametrization options.

In [3]:
max_iterations = 10000
min_error = 1.0e-7

We can use different built-in loss functions, namely `KullbackLeibler`, `Bhattacharyya`, and `SquaredHellinger`.
`KullbackLeibler` is the default loss function and the pmDAG parametrized using this loss function will be the
maximum likelihood estimation of the pmDAG.

In [4]:
pmDAG = SN2Solver(
    struct,
    sample_covariance=sample_covariance,
    loss=KullbackLeibler()
)

parametrize(pmDAG, max_iterations=max_iterations, min_error=min_error)

iteration=0          loss=9.1502         
iteration=1000       loss=0.44752        
iteration=2000       loss=0.13949        
iteration=3000       loss=0.012807       
iteration=4000       loss=2.3842e-06     


We could use the `Bhattacharyya` loss function, and it would give us the same paramterized pmDAG.


In [13]:
pmDAG = SN2Solver(
    struct,
    sample_covariance=sample_covariance,
    loss=Bhattacharyya()
)

parametrize(pmDAG, max_iterations=max_iterations, min_error=min_error)

iteration=0          loss=0.77869        
iteration=1000       loss=0.72944        
iteration=2000       loss=0.6818         
iteration=3000       loss=0.64291        
iteration=4000       loss=0.60844        
iteration=5000       loss=0.57685        
iteration=6000       loss=0.5496         
iteration=7000       loss=0.52818        
iteration=8000       loss=0.51237        
iteration=9000       loss=0.50097        
Did not converge in the maximum number of iterations!


This prints the same covariance matrix as before.


In [6]:
print(pmDAG.visible_covariance_)

tensor([[ 110.9302,   61.3989,  188.5205,  301.2141],
        [  61.3989,  138.2396,  248.1375,  307.7289],
        [ 188.5205,  248.1375,  662.2336,  849.6810],
        [ 301.2141,  307.7289,  849.6810, 1371.0049]], device='cuda:0')


## Custom Loss Function

When non of the two built-in custom functions are desirable, one could simply define an arbitrary custom loss function.
This is done by subclassing the `LossBase` class.

The signature of the loss function is as follows. We use the methods that give the
Kullback-Leibler divergence from the sample covariange. However, these methods are arbitrary.

In [7]:
class MyLoss(LossBase):
    def loss_proxy(self, visible_covariance):
        return torch.trace(self.sample_covariance_inv @ visible_covariance) - torch.logdet(visible_covariance)

    def loss(self, visible_covariance):
        return (self.loss_proxy(visible_covariance) - self.size + self.sample_covariance_logdet) / 2

    def loss_backward(self, visible_covariance, visible_covariance_grad):
        visible_covariance_grad.copy_(self.sample_covariance_inv)
        visible_covariance_grad.subtract_(torch.inverse(visible_covariance))

Notice that there are three functions to be overloaded: `loss`, `loss_proxy`, and `loss_backward`.
- `loss` computes the actual distance between the sample covariance and the induced covariance.
- `loss_proxy` is a computationally more effective function that is a monotonic counterpart of the `loss` function
- `loss_backward` computes the derivative of the distance with respect t the induced covariance.

Among these functions, only `loss_backward` is required for the parametrization to work. The other two can be used
to get the loss or loss proxy values when one desires.

Now, let's re-parametrize the pmDAG using this new custom function.

In [8]:
pmDAG = SN2Solver(
    struct,
    sample_covariance=sample_covariance,
    loss=MyLoss()
)

parametrize(pmDAG, max_iterations=max_iterations, min_error=min_error)

iteration=0          loss=3.3356         
iteration=1000       loss=0.86539        
iteration=2000       loss=0.33636        
iteration=3000       loss=0.003971       
iteration=4000       loss=5.6386e-05     
iteration=5000       loss=4.4107e-06     
iteration=6000       loss=0.00042236     


If you notice, this gives similar convergence results to our initial `KullbackLeibler` loss function.
Now, let's print the resulting visible covariance matrix.

In [9]:
print(pmDAG.visible_covariance_)

tensor([[ 2.0008,  2.9985,  5.9992,  7.9986],
        [ 2.9985,  6.9926, 11.9895, 15.9849],
        [ 5.9992, 11.9895, 22.9867, 29.9804],
        [ 7.9986, 15.9849, 29.9804, 40.9718]], device='cuda:0')
